In [ ]:
import sqlite3
import pandas as pd
import zipfile

# Extract SQLite file from ZIP archive
with zipfile.ZipFile("AmItheAsshole.sqlite.zip", "r") as zip_ref:
    zip_ref.extract("AmItheAsshole.sqlite")

# Connect to SQLite database
conn = sqlite3.connect("AmItheAsshole.sqlite")

# Retrieve table names
cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()

# Print the list of tables
print("Tables in the SQLite database:")
for table in tables:
    print(table[0])

# Close the connection
conn.close()

In [ ]:
# Brining in posts
conn = sqlite3.connect("AmItheAsshole.sqlite")

query = "SELECT * FROM submission"

if conn is not None:
    df = pd.read_sql_query(query, conn)
else:
    print("Error: Database connection is closed.")

df.to_csv('posts.csv')

conn.close()

In [ ]:
# Brining in comments
conn = sqlite3.connect("AmItheAsshole.sqlite")

query = "SELECT * FROM comment"

if conn is not None:
    df_comment = pd.read_sql_query(query, conn)
    df_comment.to_csv('comments.csv')
else:
    print("Error: Database connection is closed.")


conn.close()

In [ ]:
df_posts = pd.read_csv('posts.csv')


In [ ]:
df_comment = pd.read_csv('comments.csv')

## Reviwing existing dataset

In [ ]:
# Count duplicates in each column
def count_duplicates_in_columns(df):
    duplicate_counts = {}
    for column in df.columns:
        duplicates_count = df[df.duplicated(subset=[column], keep=False)].shape[0]
        duplicate_counts[column] = duplicates_count
    return duplicate_counts



### Posts df

In [ ]:
df_posts.head()

In [ ]:
from IPython.display import display

# Check the data types and non-null counts
display(df_posts.info())


# Get a summary of the categorical columns
display(df_posts['title'].describe())
display(df_posts['selftext'].describe())

In [ ]:
# Rename 'Unnamed: 0' to 'index'
df_posts.rename(columns={'Unnamed: 0': 'index'}, inplace=True)

# Drop duplicate rows if any
df_posts['selftext'].drop_duplicates(inplace=True)

df_posts['selftext'].describe()


Analysis of TITLE colum indicates duplicate in the title, which doesn't necceserraly mean that it is a duplicate record, as titles can overlap.
Analysis of the post body indicates we have quite a few removed posts and some duplicates, both will be removed.

In [ ]:
# Removing empty posts and duplicats
# Filter out rows where the 'selftext' column is '[removed]'
df_posts = df_posts[df_posts['selftext'] != '[removed]']

# Removing duplicate entries in the 'selftext' column
df_posts.drop_duplicates(subset='selftext', inplace=True)

# Renaming the 'selftext' column to 'body'
df_posts.rename(columns={'selftext': 'body'}, inplace=True)


In [ ]:
# reviewing the title duplicates to understand weather it's just and accidentally same selected title for different posts:
import random

duplicated_titles = df_posts[df_posts.duplicated(subset=['title'], keep=False)]['title'].unique()
random_title = random.choice(duplicated_titles)
records_with_title = df_posts[df_posts['title'] == random_title]


In [ ]:
records_with_title['body']

In [ ]:
rows_to_display = [1307, 1370]

for index in rows_to_display:
    print(records_with_title.loc[index, 'body'])
    print()  # Add a newline for clarity between rows

so it seems that the posts with the same title are very likely a duplicates

In [ ]:
#reviewed another example and deleted to not cluter this notebook

Interestingly, the story iis not a word by word copy, but it is clearly the same story. For clenliness we will drop all the rows that have duplcate title.

We have droped all suplicates in the body and explored remaining duplicates in the title. Those are also duplicates but usually with some extra sentance, for instance "Reposting from different account". Therefore we droped all the title duplicates as well.

In [ ]:
df_posts.drop_duplicates(subset='title', keep='first', inplace=True)

In [ ]:
# Count duplicates in each column
duplicate_counts = count_duplicates_in_columns(df_posts)
duplicate_counts

In [ ]:
# Drop columns 'Unnamed: 0', 'id', and 'created_utc'
columns_to_drop = ['Unnamed: 0', 'id', 'created_utc']
df_posts.drop(columns=columns_to_drop, inplace=True)

# Set 'submission_id' as the index
df_posts.set_index('submission_id', inplace=True)

# Print the first few rows to verify changes
df_posts.head()

Following the same best practices as in other AITA reaserch I'm dropping all posts that have score under 3 (under 3 number of upvotes), to ensure good quality of posts


In [ ]:
# Drop rows where 'score' is under 3
df_posts = df_posts[df_posts['score'] >= 3]

In [ ]:
df_posts['score'].describe()

### In depth body analysis

In [ ]:
import re
from nltk.corpus import stopwords

In [ ]:
import nltk
nltk.download('stopwords')

In [ ]:
# Converting to lowercase
df_posts['cleaned_body'] = df_posts['body'].str.lower()

# Removing "AITA"
df_posts['cleaned_body'] = df_posts['cleaned_body'].str.replace(r'\baita\b', '')

# Removing special characters, numbers, etc. (if needed)
df_posts['cleaned_body'] = df_posts['cleaned_body'].apply(lambda x: re.sub(r'[^a-zA-Z0-9\s]', '', x))

# Removing stopwords (example)
stop_words = set(stopwords.words('english'))
df_posts['cleaned_body'] = df_posts['cleaned_body'].apply(lambda x: ' '.join(word for word in x.split() if word not in stop_words))



In [ ]:
df_posts['word_count'] = df_posts['cleaned_body'].str.split().str.len()

In [ ]:
df_posts['word_count'].describe()

In [ ]:
# Droping posts with word count less than 50
df_posts = df_posts[df_posts['word_count'] >= 50]

In [ ]:
# Drop the first row which only contains subreddit rules
df_posts = df_posts.drop(df_posts.index[0])

In [ ]:
len(df_posts)

We removed duplicates by both body and title columns, droped all rows with score lower then 3 and with post word_cound under 50 to ensure decent quality of stories. We also removed fist record as it only contains rules of the subreaddit instaed of the post.

We have completed coule of preprocessing steps, lowecased everything, removed special characters and aita keyword.

Those changes have left us with the 30135 records.

In [ ]:
# Save the DataFrame to CSV
df_posts.to_csv('df_posts_cleaned.csv', index=True)

In [ ]:
df_posts_cleaned = pd.read_csv("df_posts_cleaned.csv")

In [ ]:
post_ids = df_posts_cleaned['submission_id'].unique()